In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv('Downloads/cyberbullying_tweets(ML).csv')
df.head()
df.isnull().sum()
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
sw = stopwords.words('English')
lm = WordNetLemmatizer()
preprocessed_texts = []
for text in df['tweet_text']:
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    tokens = [token for token in tokens if token not in sw]
    tokens = [lm.lemmatize(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)
    preprocessed_texts.append(preprocessed_text)

df['preprocessed_text'] = preprocessed_texts
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(df['preprocessed_text'])
y = df['cyberbullying_type']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
def eval_model(y_test,y_pred):
    cm=confusion_matrix(y_test,y_pred)
    cr=classification_report(y_test,y_pred)
    acc=accuracy_score(y_test,y_pred)
    print('Confusion Matrix\n',cm)
    print('Classification Report\n',cr)
    print('Accuracy Score',acc)


def model_score(model):
    print('Train Score',model.score(x_train,y_train))
    print('Test Score',model.score(x_test,y_test)) 

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


models={
    'Multinomial Naive Bayes':MultinomialNB(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier(),
    'Random Forest':RandomForestClassifier()
}


for name,model in models.items():
     model.fit(x_train, y_train)
     y_pred=model.predict(x_test)
     print(f"Model:{name}")
     print(f"Predicted Values:{y_pred}")
     model_score(model)
     eval_model(y_test,y_pred)
best_model = max(models, key=models.get)
print(f"The model with the best accuracy is: {best_model}")    

Model:Multinomial Naive Bayes
Predicted Values:['religion' 'ethnicity' 'ethnicity' ... 'age' 'ethnicity' 'religion']
Train Score 0.859906167273871
Test Score 0.7924310724394591
Confusion Matrix
 [[1628    4    1    2    2    5]
 [  39 1512    3    3   12   34]
 [  30   28 1362   92   70   23]
 [ 217   58  115  628  370  154]
 [ 203  107  135  175  833   66]
 [   4    4   11   10    3 1596]]
Classification Report
                      precision    recall  f1-score   support

                age       0.77      0.99      0.87      1642
          ethnicity       0.88      0.94      0.91      1603
             gender       0.84      0.85      0.84      1605
  not_cyberbullying       0.69      0.41      0.51      1542
other_cyberbullying       0.65      0.55      0.59      1519
           religion       0.85      0.98      0.91      1628

           accuracy                           0.79      9539
          macro avg       0.78      0.79      0.77      9539
       weighted avg       0.78  